# Validazione e ottimizzazione iperparametri

In questo capitolo cercheremo di migliorare i risultati ottenuti con i passi precedenti, ottimizzando gli algoritmi e valutando bene i risultati.

## Caricamento del dataset e utilizzo delle pipeline
In questo capitolo utilizzeremo il dataset *Breast Cancer Wisconsin* le prime due colonne sono un identificativo del caso e la diagnosi corrispondente le caratterstiche dalla 3 alla 32 sono valori oggettivi ottenuti dalle immagini digitalizzare del nucleo della cellula.

In [2]:
import pandas as pd
df = pd.read_csv('./data/data.csv')

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [17]:
X = df.loc[:,'radius_mean':'fractal_dimension_worst']
y = df.loc[:,'diagnosis']

In [18]:
label_map = {'M': 1, 'B': 0}
y = y.map(label_map)

Ora che abbiamo creato il dataset nel formato desiderato andiamo a dividere i dati in due parti, l'80% verranno utilizzati per l'addestramento (train) mentre il 20% per testare la correttezza del nostro sistema (test).

In [19]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.20)

### Combinare estimatori e trasformatori in una pipeline

Creiamo ora un sistema che esegua in un solo passo la trasformazione PCA vista nella lezione precedente e la regressione logistica.
Per svolgere questo lavoro usiamo l'oggetto `Pipeline`

In [20]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

pipeline = make_pipeline(StandardScaler(),PCA(n_components=2),LogisticRegression(solver='lbfgs'))
pipeline.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression())])

In [22]:
score = pipeline.score(X_test,y_test)
print("Test accuracy: {0:.3f}".format(score))

Test accuracy: 0.982


La funzione `make_pipeline` accetta in ingresso un numero arbitrario di transformer (che supportano sia il metodo `fit` che il metodo `transform`) seguito da un oggetto estimatore che implementa i metodi `fit` e `predict`.

Possiamo pensare che l'oggetto `Pipeline` sia una box in cui quando chiamiamo il metodo `fit` i dati passano attraverso tutti i trasformatori (viene chiamato il metodo `fit`) fino ad arrivare al trasformatore finale.

Quando invece viene chiamato il metodo `predict` i dati passano attravesrso i trasformatori utilizzando il metodo `tranform` fino ad arrivare alla fine dove incontrano l'estimatore in cui viene richiamato il metodo `predict`.

## Utilizzo della convalida incrociata k-fold per valutare le prestazioni del modello